# Using rapidfuzz to extract affiliatuons from JPE

First the JSON output file from tezseract is read in and made into a useable dataframe

In [6]:
#Importing the libraries
import json
import pandas as pd
import requests

# Import JSON file from Github repository
tesseract_input = requests.get('https://raw.githubusercontent.com/Matt970204/Affiliations_V2/Clean-online-hosted/1_Input%20files%20and%20processing%20input%20files/1_1_Input%20files/Tesseract%20affiliation%20output/JPE/JPE_affiliation_output_1940_2010.json')
json_data = tesseract_input.json()

#Use the 2 lines below for a local import of the data
#tesseract_input = open('/Users/MattJohnson/Desktop/Affiliations/1_Input files and processing input files/1_1_Input files/Tesseract affiliation output/JPE/JPE_affiliation_output_1940_2010.json')
#json_data = json.load(tesseract_input)

# Define Journal and Dataframe
Journal_Name = 'JPE'
Reference_Data_List = []

# Extract Json Data into dataframe
for key1, value1 in json_data.items():
    Article_ID = key1
    content_type = value1["content_type"]
    author_list = value1["authors"]
    authorsNo = len(author_list)
    stable_url = value1["stable_url"]
    for key2 in value1["affiliations"]:
        Ref_type = key2
        if key2 == "found":
            ref_string = value1["affiliations"][key2]
        else: 
            for key3 in value1["affiliations"]["raw"]:
                if len(value1["affiliations"]["raw"]) > 1:
                    #print(key1, "   ",len(value1["affiliations"]["raw"]))
                if key3 == "1":
                    #print(key3)
                    ref_string = value1["affiliations"][key2][key3]
                    #print(ref_string)

    Reference_Data_List.append({'Journal_Name' : Journal_Name,
    'Authors_count' : authorsNo, 
    'Authors_list' : author_list,
    'Article_ID' : Article_ID, 'content_type' : content_type,
    'stable_url' : stable_url, 
    'Ref_type': Ref_type,  'ref_string':ref_string })

# Create a dataframe from the extracted data
JPE_aff_ref_output_1940_2010 = pd.DataFrame.from_records(Reference_Data_List)
print(JPE_aff_ref_output_1940_2010.head())
print(JPE_aff_ref_output_1940_2010.shape)
#USe line below to send data to excel to view full file
#JPE_aff_ref_output_1940_2010.to_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/Working files/JPE_aff_ref_output_1940_2010.xlsx')        

1
['Preface\n', 'The papers in this issue of the /PE were presented at a Festschrift con-\nference in honor of James J. Heckman\'s 65th birthday. The conference\nattendees included many of his former graduate students, coauthors,\nand colleagues. Thirteen excellent papers were submitted to this special\nissue. They went through the regular refereeing process, and six were\nultimately accepted. Below, we describe how the papers included in this\nissue were inspired and influenced by Heckman\'s research.\n\nHeckman has produced a large body of research examining the ori-\ngins of inequality in socioeconomic outcomes. An important insight\nfrom this research is that the family has a large influence on an individ-\nual\'s human capital formation. This finding led Heckman to state that\n*the accident of birth is a major source of inequality" (2008, 289).! After\nHeckman won the Nobel Prize in 2000, a large part of his research focused\non understanding what types of policy interventions can

Next the Unique list of universities is read in, this is used as a database to check the affiliton string against using rapidfuzz. 

In [106]:
# Load in the list of unique Universites from github stored in 2_fuzzy matching
#affs = pd.read_excel("https://raw.githubusercontent.com/Matt970204/Affiliations_V2/main/2_Extracting%20affilitations/2_Fuzzy%20matching/University%20list/Unique%20Universities.xlsx")


affs = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/Unique Universities.xlsx')
print(affs.shape)

for i in range(len(affs)):
    #Replaceing title case and stripping leading space
    #affs.iat[i,0] = affs.iat[i,0].strip().title()
    affs.iloc[i]['Affilliation'] = affs.iloc[i]['Affilliation'].strip().title()
    #affs.iat[i,0] = affs.iat[i,0].strip().upper()

Affiliation_choices = affs['Affilliation'].values.tolist()

print(affs['Affilliation'].head())

(842, 14)


/var/folders/23/ft9qf9c11rj_vqhbjml910f40000gn/T/ipykernel_2675/1542333116.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  affs.iloc[i]['Affilliation'] = affs.iloc[i]['Affilliation'].strip().title()


0      Yale University
1            Cambridge
2              Harvard
3               Oxford
4    Oxford University
Name: Affilliation, dtype: object


In [108]:
#testdata = RES_1940_2010.loc[0:20].reset_index(drop = True)
testdata = JPE_aff_ref_output_1940_2010

for i in range(len(testdata)):
#for i in range(0,2):
    if testdata.iloc[i]['Ref_type'] == "raw":
        joined =  " ".join(testdata.iloc[i]['ref_string'])
        ref_string = joined.strip().title()
        clean_ref_string = re.sub('\?|\:|\"', '', ref_string)
        testdata.loc[i,['ref_string_clean1']] = clean_ref_string
    else:
        ref_string = testdata.iloc[i]['ref_string'].strip().title()
        clean_ref_string = re.sub('\?|\:|\"', '', ref_string)
        testdata.loc[i,['ref_string_clean1']] = clean_ref_string

    print(testdata.loc[i,['ref_string_clean1']])
print(testdata.columns.values)

ref_string_clean1    Johannes Hórner\n\nYale University\n\nAndrzej ...
Name: 0, dtype: object
ref_string_clean1    Gabriel Chodorow-Reich\n\nHarvard University A...
Name: 1, dtype: object
ref_string_clean1    David Gill\n\nPurdue University\n\nVictoria Pr...
Name: 2, dtype: object
ref_string_clean1    Tobias J. Klein\n\nTilburg University\n\nChris...
Name: 3, dtype: object
ref_string_clean1    David Mitch\n\nUniversity Of Maryland, Baltimo...
Name: 4, dtype: object
ref_string_clean1    Suresh Naidu\n\nColumbia University\n\nYaw Nya...
Name: 5, dtype: object
ref_string_clean1    Amanda Pallais\n\nHarvard University And Natio...
Name: 6, dtype: object
ref_string_clean1    News Shock And Granger Causality\n\nDr. Emmett...
Name: 7, dtype: object
ref_string_clean1    Eduardo M. Azevedo\n\nUniversity Of Pennsylvan...
Name: 8, dtype: object
ref_string_clean1    Manuel Bagues\n\nAalto University And Institut...
Name: 9, dtype: object
ref_string_clean1    Yeon-Koo Che\n\nColumbia University\n\n

In [109]:
#Reading in the master file and correctly formatting the article ID to match back onto the meltef file

#Load in master from Github
#AER_master = pd.read_excel("https://raw.githubusercontent.com/Matt970204/Affiliations_V2/main/1_Input%20files%20and%20processing%20input%20files/1_1_Input%20files/Master%20lists/Processed/AER_processed.xlsx")

#Local store of AER_master
JPE_master = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/1_Input files and processing input files/1_1_Input files/Master lists/Processed/JPE_processed.xlsx')
for i in range(len(JPE_master)):
    JPE_master.loc[i,['Article_ID']] = JPE_master.loc[i,['stable_url']].to_string().rsplit('/',1)[-1]

JPE_master_condenced  = JPE_master[['Article_ID','year']]
JPE_master_condenced['year'] = JPE_master_condenced['year'].astype('int')
JPE_master_condenced.head()

/var/folders/23/ft9qf9c11rj_vqhbjml910f40000gn/T/ipykernel_2675/679620167.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  JPE_master_condenced['year'] = JPE_master_condenced['year'].astype('int')


,Article_ID,year
0,26549909,2016
1,26549910,2016
2,26549911,2016
3,26549912,2016
4,26549913,2016


In [110]:
# MErging the msterfile onto the new melted file to get additional data
testdata_merged = testdata.merge(JPE_master_condenced, how = 'left', on = 'Article_ID')
testdata_merged = testdata_merged.sort_values(by=['year'])
testdata = testdata_merged.reset_index(drop = True)

In [111]:
# Excel output to get an overview of the data
testdata.head
testdata.to_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/JPE/Test files/JPE_aff_ref_output_1940_2010_affs_out setup.xlsx') 

In [112]:
import re
from rapidfuzz import fuzz
import math

for x in range(len(testdata)):
    # Read in authors
    authors = testdata.iloc[x]['Authors_list']
    str1 = testdata.iloc[x]['ref_string_clean1']

    # Split on new line as well as don't include blank lines
    ref_split = re.split(r"\n",str1 )
    while("" in ref_split):
        ref_split.remove("")


#Finding the authors name and corrosponding affiliations
    count_found = 0
    #Loop through author names in author list
    for d in range(len(authors)):
        #Loop through the data that is split into possible authors affiliations
        for j in range(len(ref_split)):
            #Check if the author name is in the split string
            ratio_author  = fuzz.partial_ratio(ref_split[j], authors[d])
            # Confiring if author name is in the affiliaton
            if ratio_author > 93:
                count_found = count_found +1
                #append the next line of the document to a column for each author
                if j+1 < len(ref_split):
                    testdata.loc[x, str(d)+'_author'] = authors[d]
                    # checking for short erroenous splits, if a line has a article on it
                    if len(ref_split[j+1]) > 3:
                        testdata.loc[x, str(d)+'_author_affs'] = ref_split[j+1]
                    else:
                        testdata.loc[x, str(d)+'_author_affs'] = ''
            # Prevents double finding of author name, sometimes name occurs twice 
            # but first occurance is the affiliation
            if count_found == len(authors):
                break
                #else:


In [113]:
# Creating a dictionary for consolidation
# Read in from Excel sheet, 2 columns first one with incorrect spelling and second one with correct spellig

cities = pd.read_excel('/Users/MattJohnson/Library/CloudStorage/OneDrive-UniversityofCapeTown/Masters_thesis/Data checks/consolidated list 09 11.xlsx', 'Cities')
#cities = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Cities')
cities = cities[['Incorrect spelling', 'Correct spelling']]

misspellings = pd.read_excel('/Users/MattJohnson/Library/CloudStorage/OneDrive-UniversityofCapeTown/Masters_thesis/Data checks/consolidated list 09 11.xlsx', 'Misspellings')
#misspellings = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Misspellings')
misspellings = misspellings[['Incorrect spelling', 'Correct spelling']]

comma_in_name = pd.read_excel('/Users/MattJohnson/Library/CloudStorage/OneDrive-UniversityofCapeTown/Masters_thesis/Data checks/consolidated list 09 11.xlsx', 'Comma in name')
#comma_in_name = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Comma in name')
comma_in_name = comma_in_name[['Incorrect spelling', 'Correct spelling']]

colleges_full = pd.read_excel('/Users/MattJohnson/Library/CloudStorage/OneDrive-UniversityofCapeTown/Masters_thesis/Data checks/consolidated list 09 11.xlsx', 'Colleges Full')
#colleges_full = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Colleges Full')
colleges_full = colleges_full[['Incorrect spelling', 'Correct spelling']]

colleges = pd.read_excel('/Users/MattJohnson/Library/CloudStorage/OneDrive-UniversityofCapeTown/Masters_thesis/Data checks/consolidated list 09 11.xlsx', 'Colleges')
#colleges = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Colleges')

correct = pd.read_excel('/Users/MattJohnson/Library/CloudStorage/OneDrive-UniversityofCapeTown/Masters_thesis/Data checks/consolidated list 09 11.xlsx', 'Correct')
#correct = pd.read_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/University list/consolidated list.xlsx', 'Correct')

frames = [cities, misspellings, comma_in_name,colleges_full ,colleges,correct]
consolidated_affiliations = pd.concat(frames)
print(consolidated_affiliations.shape)

for i in range(len(consolidated_affiliations)):
    #Replaceing title case and stripping leading space
    #affs.iat[i,0] = affs.iat[i,0].strip().title()
    consolidated_affiliations.iloc[i]['Incorrect spelling'] = consolidated_affiliations.iloc[i]['Incorrect spelling'].strip().title()
    #affs.iat[i,0] = affs.iat[i,0].strip().upper()

consolidated_affiliations_dict = consolidated_affiliations.set_index('Incorrect spelling').to_dict()['Correct spelling']
print(consolidated_affiliations_dict)

(1475, 2)
{'Auckland, Nz': 'University of Auckland', 'Auckland, New Zealand': 'University of Auckland', 'Ames, Iowa': 'Iowa State University', 'Ames, Iowa, Usa': 'Iowa State University', 'Washington, Dc': 'Washington DC (City)', 'Cambridge, Massachusetts': 'Cambridge Massachusetts (City)', 'Chicago, Illinois': 'Chicago Illinois (City)', 'Chicago, Illinoi': 'University Of Illinois Chicago', 'Jerusalem, Israel': 'Jerusalem Israel (City)', 'Manchester, England': 'Manchester England (City)', 'Chicago, Illinois, Usa': 'Chicago Illinois (City)', 'Mountain Lakes, New Jersey': 'Mountain Lakes, New Jersey (City)', 'Antwerp': 'Antwerp Belguim (City)', 'Vienna': 'Vienna Austria (City)', 'Baghdad': 'Baghdad Iraq (City)', 'Long Island City': 'Long Island City (City)', 'Brooklyn, New York': 'Brooklyn, New York (City)', 'Wayland, Massachusetts': 'Wayland, Massachusetts (City)', 'Hanover, New Hampshire': 'Hanover, New Hampshire (City)', 'Bell Laboratories, Inc': 'Bell Laboratories', 'Board Of Governor

In [114]:
key  = 'New York University'
key  = 'University Of California, Berkeley'


if key in consolidated_affiliations_dict:
    print(key)

University Of California, Berkeley


In [115]:
import numpy as np
testdata = testdata.replace(np.nan, '', regex=True)
testdata["0_author_affs_consolidated"] = ""
testdata["1_author_affs_consolidated"] = ""
testdata["2_author_affs_consolidated"] = ""
testdata["3_author_affs_consolidated"] = ""
testdata["4_author_affs_consolidated"] = ""
testdata["5_author_affs_consolidated"] = ""
testdata["0_author_affs_consolidated_0"] = ""
testdata["0_author_affs_consolidated_1"] = ""
testdata["0_author_affs_consolidated_2"] = ""
testdata["1_author_affs_consolidated_1"] = ""
testdata["1_author_affs_consolidated_2"] = ""
testdata["4_author_affs"] = ""
testdata["5_author_affs"] = ""

for i in range(len(testdata)):
#for i in range(3430,3431):
#for i in range(3715,3716):
    authors = testdata.iloc[i]['Authors_list']
    for auth_no in range(len(authors)):
        #print(testdata.iloc[i]['0_author_affs'])
        testdata.loc[i,str(auth_no)+'_author_affs'] =  re.sub(r'\.','',testdata.iloc[i][str(auth_no)+'_author_affs']).title()
        testdata.loc[i,str(auth_no)+'_author_affs'] =  re.sub(r'The ','',testdata.iloc[i][str(auth_no)+'_author_affs']).title()
        #print(testdata.iloc[i]['0_author_affs'])
        list = []

    #Loop through the list of incorrect spellings and replace with corrected spellings
        for key in consolidated_affiliations_dict.keys():
            if key in testdata.iloc[i][str(auth_no)+'_author_affs']:
                list.append(key)
                print((key))
                print(list)

    #If there is more than 1 match in the consolidation it needs to match on the longer one, solves for college issue
                if len(list) <= 1:
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated']  = re.sub(key, consolidated_affiliations_dict[key],testdata.iloc[i][str(auth_no)+'_author_affs']).strip().rstrip(',')
                else:
                    print(consolidated_affiliations_dict[list[0]])
                    print(consolidated_affiliations_dict[list[1]])
                    # For colleges
                    if(len(consolidated_affiliations_dict[list[0]]) == len(consolidated_affiliations_dict[list[1]])):
                        #print("HERE")
                        if len(list[0]) > len(list[1]):
                            print("If")
                            testdata.loc[i,str(auth_no)+'_author_affs_consolidated']  = re.sub(list[0], consolidated_affiliations_dict[list[0]],testdata.iloc[i][str(auth_no)+'_author_affs']).strip().rstrip(',')
                            #print(list[0])
                        # Full  University Of Illinois System Urbana-Champaign issue
                        elif len(list[1]) > len(list[0]):
                            print("Elif")
                            testdata.loc[i,str(auth_no)+'_author_affs_consolidated']  = re.sub(list[1], consolidated_affiliations_dict[list[1]],testdata.iloc[i][str(auth_no)+'_author_affs']).strip().rstrip(',')
                        else:
                            print("else")
                            first_fix = re.sub(list[1], consolidated_affiliations_dict[list[1]],testdata.iloc[i][str(auth_no)+'_author_affs']).strip().rstrip(',') 
                            testdata.loc[i,str(auth_no)+'_author_affs_consolidated']  = re.sub(key, consolidated_affiliations_dict[key],first_fix).strip().rstrip(',') 
                    else:
                        first_fix = re.sub(list[0], consolidated_affiliations_dict[list[0]],testdata.iloc[i][str(auth_no)+'_author_affs']).strip().rstrip(',') 
                        #print(first_fix)
                        testdata.loc[i,str(auth_no)+'_author_affs_consolidated']  = re.sub(key, consolidated_affiliations_dict[key],first_fix).strip().rstrip(',') 
        print(i)
        print('Full ',testdata.loc[i,str(auth_no)+'_author_affs_consolidated'])
        #else: 
        #    print("non existant")
        auths_affs_split_And = re.split(' And ', testdata.iloc[i][str(auth_no)+'_author_affs_consolidated'])
        #print(auths_affs_split_And)
        #print(len(auths_affs_split_And))

        for aff in range(len(auths_affs_split_And)):
            testdata.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(aff)]  = auths_affs_split_And[aff].strip().rstrip(',') 
            print('Auth', str(auth_no), str(aff) , testdata.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(aff)])

        auths_affs_split_comma = re.split(',', testdata.iloc[i][str(auth_no)+'_author_affs_consolidated_0'])
        #print(auths_affs_split_And)
        #print(len(auths_affs_split_And))
        if len(auths_affs_split_comma) == 2:
            for affcomma in range(len(auths_affs_split_comma)):
                testdata.loc[i,str(auth_no)+'_author_affs_consolidated_0']  = auths_affs_split_comma[0].strip().rstrip(',')
                if  testdata.loc[i,str(auth_no)+'_author_affs_consolidated_1'] == '':
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated_1']  = auths_affs_split_comma[1].strip().rstrip(',')
                else:
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated_2']  = auths_affs_split_comma[1].strip().rstrip(',')
                print('Auth', str(auth_no), str(affcomma) , testdata.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(affcomma)])
                print(testdata.loc[i,str(auth_no)+'_author_affs_consolidated_2'])
        if len(auths_affs_split_comma) == 3:
            for affcomma in range(len(auths_affs_split_comma)):
                testdata.loc[i,str(auth_no)+'_author_affs_consolidated_0']  = auths_affs_split_comma[0].strip().rstrip(',')
                if  testdata.loc[i,str(auth_no)+'_author_affs_consolidated_1'] == '':
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated_1']  = auths_affs_split_comma[1].strip().rstrip(',')
                else:
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated_2']  = auths_affs_split_comma[1].strip().rstrip(',')
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated_3']  = auths_affs_split_comma[2].strip().rstrip(',')
                print('Auth', str(auth_no), str(affcomma) , testdata.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(affcomma)])
                print(testdata.loc[i,str(auth_no)+'_author_affs_consolidated_2'])
        if len(auths_affs_split_comma) == 4:
            for affcomma in range(len(auths_affs_split_comma)):
                testdata.loc[i,str(auth_no)+'_author_affs_consolidated_0']  = auths_affs_split_comma[0].strip().rstrip(',')
                if  testdata.loc[i,str(auth_no)+'_author_affs_consolidated_1'] == '':
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated_1']  = auths_affs_split_comma[1].strip().rstrip(',')
                else:
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated_2']  = auths_affs_split_comma[1].strip().rstrip(',')
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated_3']  = auths_affs_split_comma[2].strip().rstrip(',')
                    testdata.loc[i,str(auth_no)+'_author_affs_consolidated_4']  = auths_affs_split_comma[3].strip().rstrip(',')

                print('Auth', str(auth_no), str(affcomma) , testdata.loc[i,str(auth_no)+'_author_affs_consolidated_'+str(affcomma)])
                print(testdata.loc[i,str(auth_no)+'_author_affs_consolidated_2'])

    print(i)

Washington, Dc
['Washington, Dc']
0
Full  Washington DC (City)
Auth 0 0 Washington DC (City)
0
New York University
['New York University']
York University
['New York University', 'York University']
New York University
York University
1
Full  New York University
Auth 0 0 New York University
1
University Of California, Berkeley
['University Of California, Berkeley']
University Of California
['University Of California, Berkeley', 'University Of California']
UCB
University Of California System
2
Full  UCB
Auth 0 0 UCB
University Of California, Berkeley
['University Of California, Berkeley']
University Of California
['University Of California, Berkeley', 'University Of California']
UCB
University Of California System
2
Full  UCB
Auth 1 0 UCB
2
New York University
['New York University']
York University
['New York University', 'York University']
New York University
York University
3
Full  New York University
Auth 0 0 New York University
3
University Of Minnesota
['University Of Minnesota']
4

In [116]:
print(testdata.columns.values)

['Journal_Name' 'Authors_count' 'Authors_list' 'Article_ID' 'content_type'
 'stable_url' 'Ref_type' 'ref_string' 'ref_string_clean1' 'year'
 '0_author' '0_author_affs' '1_author' '1_author_affs' '2_author'
 '2_author_affs' '3_author' '3_author_affs' '4_author' '4_author_affs'
 '5_author' '5_author_affs' '0_author_affs_consolidated'
 '1_author_affs_consolidated' '2_author_affs_consolidated'
 '3_author_affs_consolidated' '4_author_affs_consolidated'
 '5_author_affs_consolidated' '0_author_affs_consolidated_0'
 '0_author_affs_consolidated_1' '0_author_affs_consolidated_2'
 '1_author_affs_consolidated_1' '1_author_affs_consolidated_2'
 '1_author_affs_consolidated_0' '2_author_affs_consolidated_0'
 '3_author_affs_consolidated_0' '0_author_affs_consolidated_3'
 '2_author_affs_consolidated_1' '2_author_affs_consolidated_2'
 '1_author_affs_consolidated_3' '2_author_affs_consolidated_3'
 '3_author_affs_consolidated_1' '4_author_affs_consolidated_0'
 '0_author_affs_consolidated_4' '3_author_affs

In [117]:
testdata = testdata[['Journal_Name' ,'Authors_count', 'Authors_list', 'Article_ID', 'content_type',
 'stable_url', 'Ref_type' ,'ref_string', 'ref_string_clean1', 'year',
 '0_author_affs', 
 '0_author_affs_consolidated',
 '0_author_affs_consolidated_0',
 '0_author_affs_consolidated_1', 
 '0_author_affs_consolidated_2', 
# '0_author_affs_consolidated_3',
 '1_author_affs', 
 '1_author_affs_consolidated',
 '1_author_affs_consolidated_0',
 '1_author_affs_consolidated_1',
  '1_author_affs_consolidated_2',
 '2_author_affs',
 '2_author_affs_consolidated',
 '2_author_affs_consolidated_0',
 '2_author_affs_consolidated_1', 
 '2_author_affs_consolidated_2' ,
 '3_author_affs',
 '3_author_affs_consolidated', 
 '3_author_affs_consolidated_0',
 '3_author_affs_consolidated_1',
 '4_author_affs',
 '4_author_affs_consolidated',
 '4_author_affs_consolidated_0',
 '5_author_affs', 
 '5_author_affs_consolidated']]

In [118]:
# Excel output to get an overview of the data
testdata.head
testdata.to_excel('/Users/MattJohnson/Desktop/Affiliations/2_Extracting affilitations/2_Fuzzy matching/JPE/Test files/JPE_aff_ref_output_1940_2010_affs_out authors.xlsx') 